Setelah pada submodul sebelumnya kita membahas bagaimana mengembangkan model jst sequential untuk masalah regresi sederhana, kita akan lanjut mengembangkan model jst untuk klasifikasi biner. 

Nah, pada submodul ini kita akan coba melatih model untuk mengklasifikasikan apakah sebuah buah merupakan jeruk atau anggur. Untuk datasetnya bisa Anda unduh di [tautan berikut](https://www.kaggle.com/joshmcadams/oranges-vs-grapefruit). Sudah tidak sabar bukan. Setelah datanya Anda unduh, jangan lupa buka Colaboratory dan unggah datasetnya ke dalam Colab. Jika Anda lupa caranya, bisa lihat kembali tutorial di modul pertama Belajar Machine Learning untuk Pemula ya.

Pada cell pertama di Colab apa lagi kalau bukan impor library yang harus kita lakukan. Karena dataset kita bertipe csv, kita akan menggunakan pandas untuk merubah dataset menjadi sebuah dataframe. Untuk melakukannya, gunakan fungsi `read_csv()`.

In [2]:
import pandas as pd

df = pd.read_csv('/content/sample_data/citrus.csv')

Kemudian untuk melihat overview dari dataset kita bisa memanggil fungsi `info()` pada dataframe. Dapat kita lihat di bawah bahwa pada dataset terdapat 6 buah kolom dengan 1 kolom memiliki value bertipe ‘object’, dan 5 bertipe numerik.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


Selanjutnya gunakan fungsi `head()` untuk menampilkan dataframe kita. Pada output dari cell di bawah, label dari dataset adalah kolom pertama yang berisi string ‘orange’ dan ‘grapefruit’. 

In [6]:
df.head(10000)

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


Nah, masih ingat bukan kalau sebuah jst tidak bisa memproses string? Karena itulah kita harus mengubah nilai-nilai pada kolom label menjadi numerik terlebih dahulu agar bisa diproses oleh jst.

Jalankan kode di bawah untuk mengubah nilai-nilai pada kolom name menjadi bilangan numerik. Untuk nilai yang berisi string ‘orange’ akan diubah menjadi 0 dan nilai yang berisi string ‘grapefruit’ akan diubah menjadi 1.

In [7]:
df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Sampai pada tahap ini model kita belum dapat memproses dataset ini karena dataset kita masih dalam bentuk dataframe. Betul, dataset harus dalam bentuk array agar dapat diproses oleh model. Nah untungnya kita dapat melakukan ini dengan mudah menggunakan atribut values dari dataframe. Values mengembalikan numpy array yang dikonversi dari dataframe.

In [8]:
dataset = df.values

Untuk melihat dataframe yang telah diubah menjadi array, Anda dapat mengeksekusi objek `dataset` yang menampung array tersebut.

In [9]:
dataset

array([[0, 2.96, 86.76, 172, 85, 2],
       [0, 3.91, 88.05, 166, 78, 3],
       [0, 4.42, 95.17, 156, 81, 2],
       ...,
       [1, 15.59, 256.5, 168, 82, 20],
       [1, 15.92, 260.14, 142, 72, 11],
       [1, 16.45, 261.51, 152, 74, 2]], dtype=object)

Kemudian kita perlu memisahkan antara atribut dan label pada dataset. Dari eksplorasi data kita sebelumnya, dapat diketahui bahwa 4 kolom terakhir adalah kolom atribut. Untuk memilih 4 kolom terakhir jalankan kode di bawah. 

In [10]:
# pilih 4 kolom terakhir sebagai atribut
X = dataset[:,1:6]
# bilangan sebelum koma untuk memilih baris pada dataframe
# bilangan setelah koma untuk memilih kolom pada dataframe

Jangan lupa untuk memisahkan label dari dataset. Pada dataset kita, label terdapat pada kolom pertama. Jalankan kode di bawah untuk membuat sebuah array numpy yang hanya berisi label kita.

In [11]:
y = dataset[:, 0]

Selanjutnya ada tahap yang sangat penting dilakukan agar jst bisa mempelajari dataset dengan baik, apalagi kalau bukan normalisasi. Kita bisa menggunakan fungsi `fit_transform() `dari sebuah objek **MinMaxScaler** dari library **preprocessing** **SKLearn** untuk menormalisasi data kita.

In [12]:
# Normalization
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

X_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

Setelah kita memiliki atribut dan label yang terpisah, kita juga akan memisahkan kembali data kita menjadi data latih dan data uji untuk mengevaluasi kinerja model kita. Fungsi `train_test_split` dapat membantu kita untuk melakukan hal ini dengan mudah. Pada latihan ini kita akan menggunakan 30% dari seluruh data sebagai data uji.

In [17]:
# pisahkan data training dan testing
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

Dataset telah dipisahkan ke dalam training dan test set.  Karena tadi kita mengubah nilai-nilai pada kolom name menjadi bilangan numerik, yang artinya kita mengubah label menjadi tipe data boolean, maka kita perlu mengubah tipe data tersebut menjadi `float32` dengan cara berikut.

In [18]:
import numpy as np
 
Y_train = Y_train.astype(np.float32)
Y_test = Y_test.astype(np.float32)

Nah, tahap memproses data kita telah selesai. Sekarang kita mulai membangun model jst kita. Impor library di bawah dan lanjutkan ke tahap berikutnya.

In [21]:
from keras.models import Sequential
from keras.layers import Dense

Untuk model yang kita kembangkan adalah model **sequential yang memiliki 3 buah layer** seperti di bawah. *Activation function* pada 2 layer pertama yang dapat digunakan adalah `relu` untuk latihan ini. Anda dapat bereksplorasi menggunakan *activation function* lain. Untuk layer terakhir, isi parameter `unit` isi dengan 1 di mana output dari jst kita merupakan satu buah bilangan numerik. *Activation function* pada layer terakhir dipilih `sigmoid` karena `sigmoid` memetakan probabilitas dari 0 sampai 1. `Sigmoid` sangat cocok digunakan pada masalah klasifikasi biner.

In [22]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(5,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid'),
])

Kemudian setelah arsitektur dari jst dibentuk, kita perlu menentukan optimizer dan loss function dari model kita. Untuk optimizer kita akan menggunakan `stochastic gradient descent (sgd)` yang merupakan optimizer yang sangat umum dan cocok dipakai pada dataset yang berukuran kecil. Dan untuk loss yang sesuai adalah `‘binary_crossentropy’` karena masalah pada latihan kita kali ini adalah masalah 2 kelas (biner). 

Selain itu jika kita ingin menampilkan akurasi pada setiap proses pelatihan model kita dapat menambahkan parameter metrics dan mengisinya dengan string ‘accuracy’.

In [23]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Ketika semua sudah selesai, kita dapat memulai pelatihan model kita dengan menjalankan kode di bawah. Berapa akurasi yang model Anda dapatkan?

In [24]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.6670 - accuracy: 0.8411
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.6253 - accuracy: 0.8867
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5645 - accuracy: 0.9090
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4839 - accuracy: 0.9174
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3922 - accuracy: 0.9214
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3135 - accuracy: 0.9243
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2599 - accuracy: 0.9246
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2279 - accuracy: 0.9269
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2086 - accuracy: 0.9261
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1969 - accura

Terakhir, jangan lupa untuk mengevaluasi model Anda dan melihat apakah model Anda *underfit* atau *overfit*. Untuk melihat loss dan akurasi model pada data test, gunakan fungsi evaluate pada model. Fungsi `Evaluate` mengembalikan 2 nilai. Yang pertama adalah nilai loss, dan yang kedua adalah nilai akurasinya.

In [26]:
model.evaluate(X_test, Y_test)
# elemen pertama adalah loss dan elemen kedua adalah akurasi

94/94 [==============================] - 0s 1ms/step - loss: 0.1868 - accuracy: 0.9280


[0.1868325024843216, 0.9279999732971191]